# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

In [56]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import scipy.stats as stats
import numpy as np


In [90]:
# Importing the saved CSV file into dataframe 
# Imported this dataframe that was created in our EDA notebook
flights_df = pd.read_csv("/Users/ckz/Desktop/DATA_SCIENCE_BOOTCAMP/Midterm-project/flights_df.csv")

# print the first few rows of the DataFrame
flights_df.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,first_dep_time,total_add_gtime,longest_add_gtime,no_name,month,dep_hour,arr_hour,taxi_time,origin_state,flight_distance_category
0,2018-01-01,B6,B6,B6,880,B6,N794JB,880,12954,LGB,...,NaN,NaN,NaN,NaN,1,7,8,83.0,CA,SHORT
1,2018-01-01,B6,B6,B6,885,B6,N337JB,885,12478,JFK,...,NaN,NaN,NaN,NaN,1,7,9,120.0,NY,SHORT
2,2018-01-01,B6,B6,B6,886,B6,N337JB,886,14492,RDU,...,NaN,NaN,NaN,NaN,1,10,11,102.0,NC,SHORT
3,2018-01-01,B6,B6,B6,889,B6,N318JB,889,11278,DCA,...,NaN,NaN,NaN,NaN,1,18,19,162.0,DC,SHORT
4,2018-01-01,B6,B6,B6,891,B6,N348JB,891,10721,BOS,...,NaN,NaN,NaN,NaN,1,6,9,317.0,MA,MEDIUM


In [91]:
flights_df.shape

(1559045, 48)

In [92]:
# # Shuffle the data in the DataFrame and create a sample with half the size
sample_size = len(flights_df) // 10

flights_sample = flights_df.sample(n=sample_size, random_state=42)

## Reset the index of the shuffled sample

flights_sample.reset_index(drop=True, inplace=True)

In [93]:
flights_sample.shape

(155904, 48)

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [94]:
flights_sample.columns.tolist()

['fl_date',
 'mkt_unique_carrier',
 'branded_code_share',
 'mkt_carrier',
 'mkt_carrier_fl_num',
 'op_unique_carrier',
 'tail_num',
 'op_carrier_fl_num',
 'origin_airport_id',
 'origin',
 'origin_city_name',
 'dest_airport_id',
 'dest',
 'dest_city_name',
 'crs_dep_time',
 'dep_time',
 'dep_delay',
 'taxi_out',
 'wheels_off',
 'wheels_on',
 'taxi_in',
 'crs_arr_time',
 'arr_time',
 'arr_delay',
 'cancelled',
 'cancellation_code',
 'diverted',
 'dup',
 'crs_elapsed_time',
 'actual_elapsed_time',
 'air_time',
 'flights',
 'distance',
 'carrier_delay',
 'weather_delay',
 'nas_delay',
 'security_delay',
 'late_aircraft_delay',
 'first_dep_time',
 'total_add_gtime',
 'longest_add_gtime',
 'no_name',
 'month',
 'dep_hour',
 'arr_hour',
 'taxi_time',
 'origin_state',
 'flight_distance_category']

In [95]:
flights_df.shape

(1559045, 48)

### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

In [96]:
flights_df['avg_monthly_arr_delay'] = flights_df.groupby('month')['arr_delay'].transform('mean')

In [97]:
flights_df.isnull().sum()

fl_date                           0
mkt_unique_carrier                0
branded_code_share                0
mkt_carrier                       0
mkt_carrier_fl_num                0
op_unique_carrier                 0
tail_num                          0
op_carrier_fl_num                 0
origin_airport_id                 0
origin                            0
origin_city_name                  0
dest_airport_id                   0
dest                              0
dest_city_name                    0
crs_dep_time                      0
dep_time                          0
dep_delay                         0
taxi_out                          0
wheels_off                        0
wheels_on                         0
taxi_in                           0
crs_arr_time                      0
arr_time                          0
arr_delay                         0
cancelled                         0
cancellation_code           1559045
diverted                          0
dup                         

In [98]:
# This feature creates the avg taxi times for the arrival hour 
flights_df['avg_taxi_times_arr'] = flights_df.groupby('arr_hour')['taxi_time'].transform('mean')

In [99]:
flights_df.shape

(1559045, 50)

In [100]:
flights_df['fl_date'].dtype

dtype('O')

In [102]:
# Group the flights data by origin airport and count the number of flights for each airport
airport_counts = flights_df.groupby('origin')['flights'].transform('count')

# Determine the thresholds for each category based on the quartiles
thresholds = [0, airport_counts.quantile(0.25), airport_counts.quantile(0.5), airport_counts.quantile(0.75), np.inf]

# Define the categories
categories = ['Not Busy', 'Moderate', 'Busy', 'Very Busy']

# Assign the traffic category based on the number of flights from each airport
flights_df['traffic'] = pd.cut(airport_counts, bins=thresholds, labels=categories, right=False)

In [103]:
flights_df['traffic'].isnull().sum()

0

In [104]:
flights_df.tail(20)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,no_name,month,dep_hour,arr_hour,taxi_time,origin_state,flight_distance_category,avg_monthly_arr_delay,avg_taxi_times_arr,traffic
1559025,2019-07-31,DL,DL,DL,2882,DL,N377DE,2882,14570,RNO,...,NaN,7,6,8,200.0,NV,SHORT,9.147818,126.377344,Not Busy
1559026,2019-07-31,DL,DL,DL,2883,DL,N815DN,2883,10299,ANC,...,NaN,7,7,12,399.0,AK,MEDIUM,9.147818,159.242403,Not Busy
1559027,2019-07-31,DL,DL,DL,2885,DL,N356NW,2885,10423,AUS,...,NaN,7,6,8,193.0,TX,LONG,9.147818,126.377344,Moderate
1559028,2019-07-31,DL,DL,DL,2886,DL,N372DA,2886,10693,BNA,...,NaN,7,6,9,275.0,TN,LONG,9.147818,146.300733,Moderate
1559029,2019-07-31,DL,DL,DL,2887,DL,N717JL,2887,11292,DEN,...,NaN,7,14,16,173.0,CO,MEDIUM,9.147818,188.762904,Very Busy
1559030,2019-07-31,DL,DL,DL,2888,DL,N336NB,2888,10423,AUS,...,NaN,7,8,10,122.0,TX,MEDIUM,9.147818,154.643609,Moderate
1559031,2019-07-31,DL,DL,DL,2889,DL,N3749D,2889,14869,SLC,...,NaN,7,20,21,41.0,UT,MEDIUM,9.147818,150.908933,Moderate
1559032,2019-07-31,DL,DL,DL,2890,DL,N967DL,2890,10397,ATL,...,NaN,7,22,22,-13.0,GA,SHORT,9.147818,101.631200,Very Busy
1559033,2019-07-31,DL,DL,DL,2891,DL,N3751B,2891,15304,TPA,...,NaN,7,12,14,178.0,FL,MEDIUM,9.147818,178.305499,Moderate
1559034,2019-07-31,DL,DL,DL,2892,DL,N910DL,2892,13244,MEM,...,NaN,7,7,9,194.0,TN,SHORT,9.147818,146.300733,Not Busy


In [105]:
states = {
        'AK': 'O',
        'AL': 'S',
        'AR': 'S',
        'AS': 'O',
        'AZ': 'W',
        'CA': 'W',
        'CO': 'W',
        'CT': 'N',
        'DC': 'N',
        'DE': 'N',
        'FL': 'S',
        'GA': 'S',
        'GU': 'O',
        'HI': 'O',
        'IA': 'M',
        'ID': 'W',
        'IL': 'M',
        'IN': 'M',
        'KS': 'M',
        'KY': 'S',
        'LA': 'S',
        'MA': 'N',
        'MD': 'N',
        'ME': 'N',
        'MI': 'W',
        'MN': 'M',
        'MO': 'M',
        'MP': 'O',
        'MS': 'S',
        'MT': 'W',
        'NA': 'O',
        'NC': 'S',
        'ND': 'M',
        'NE': 'W',
        'NH': 'N',
        'NJ': 'N',
        'NM': 'W',
        'NV': 'W',
        'NY': 'N',
        'OH': 'M',
        'OK': 'S',
        'OR': 'W',
        'PA': 'N',
        'PR': 'O',
        'RI': 'N',
        'SC': 'S',
        'SD': 'M',
        'TN': 'S',
        'TX': 'S',
        'UT': 'W',
        'VA': 'S',
        'VI': 'O',
        'VT': 'N',
        'WA': 'W',
        'WI': 'M',
        'WV': 'S',
        'WY': 'W'
}

region_mapping = {
    'O': 'Other',
    'S': 'South',
    'W': 'West',
    'N': 'North',
    'M': 'Midwest'
}

# Clean the 'state_code' values by converting to strings and removing leading/trailing whitespaces
flights_df['orgin_state'] = flights_df['origin_state'].astype(str).str.strip()

# Create a new column 'Region' based on the mapping
flights_df['Region'] = flights_df['origin_state'].map(states).map(region_mapping)

In [106]:
flights_df.tail(20)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dep_hour,arr_hour,taxi_time,origin_state,flight_distance_category,avg_monthly_arr_delay,avg_taxi_times_arr,traffic,orgin_state,Region
1559025,2019-07-31,DL,DL,DL,2882,DL,N377DE,2882,14570,RNO,...,6,8,200.0,NV,SHORT,9.147818,126.377344,Not Busy,NV,West
1559026,2019-07-31,DL,DL,DL,2883,DL,N815DN,2883,10299,ANC,...,7,12,399.0,AK,MEDIUM,9.147818,159.242403,Not Busy,AK,Other
1559027,2019-07-31,DL,DL,DL,2885,DL,N356NW,2885,10423,AUS,...,6,8,193.0,TX,LONG,9.147818,126.377344,Moderate,TX,South
1559028,2019-07-31,DL,DL,DL,2886,DL,N372DA,2886,10693,BNA,...,6,9,275.0,TN,LONG,9.147818,146.300733,Moderate,TN,South
1559029,2019-07-31,DL,DL,DL,2887,DL,N717JL,2887,11292,DEN,...,14,16,173.0,CO,MEDIUM,9.147818,188.762904,Very Busy,CO,West
1559030,2019-07-31,DL,DL,DL,2888,DL,N336NB,2888,10423,AUS,...,8,10,122.0,TX,MEDIUM,9.147818,154.643609,Moderate,TX,South
1559031,2019-07-31,DL,DL,DL,2889,DL,N3749D,2889,14869,SLC,...,20,21,41.0,UT,MEDIUM,9.147818,150.908933,Moderate,UT,West
1559032,2019-07-31,DL,DL,DL,2890,DL,N967DL,2890,10397,ATL,...,22,22,-13.0,GA,SHORT,9.147818,101.631200,Very Busy,GA,South
1559033,2019-07-31,DL,DL,DL,2891,DL,N3751B,2891,15304,TPA,...,12,14,178.0,FL,MEDIUM,9.147818,178.305499,Moderate,FL,South
1559034,2019-07-31,DL,DL,DL,2892,DL,N910DL,2892,13244,MEM,...,7,9,194.0,TN,SHORT,9.147818,146.300733,Not Busy,TN,South


In [108]:
flight_df = flights_df[['fl_date','mkt_unique_carrier','avg_taxi_times_arr','avg_monthly_arr_delay','flight_distance_category','taxi_time','arr_hour','month','distance','air_time','traffic','Region']]



In [109]:
flight_df.head(20)

,fl_date,mkt_unique_carrier,avg_taxi_times_arr,avg_monthly_arr_delay,flight_distance_category,taxi_time,arr_hour,month,distance,air_time,traffic,Region
0,2018-01-01,B6,126.377344,3.517748,SHORT,83.0,8,1,231.0,43.0,Not Busy,West
1,2018-01-01,B6,146.300733,3.517748,SHORT,120.0,9,1,427.0,80.0,Moderate,North
2,2018-01-01,B6,159.450976,3.517748,SHORT,102.0,11,1,427.0,62.0,Moderate,South
3,2018-01-01,B6,174.078148,3.517748,SHORT,162.0,19,1,444.0,82.0,Busy,North
4,2018-01-01,B6,146.300733,3.517748,MEDIUM,317.0,9,1,1185.0,197.0,Busy,North
5,2018-01-01,B6,170.062250,3.517748,MEDIUM,216.0,13,1,1185.0,136.0,Moderate,South
6,2018-01-01,B6,154.643609,3.517748,MEDIUM,238.0,10,1,1069.0,158.0,Moderate,North
7,2018-01-01,B6,170.062250,3.517748,MEDIUM,304.0,13,1,1180.0,184.0,Not Busy,North
8,2018-01-01,B6,170.038468,3.517748,MEDIUM,222.0,18,1,1180.0,142.0,Not Busy,South
9,2018-01-01,B6,-1384.253723,3.517748,MEDIUM,-2128.0,0,1,1065.0,152.0,Busy,North


In [ ]:
# # Shuffle the data in the DataFrame and create a sample with half the size
sample_size = len(flight_df) // 10

flights_sample = flight_df.sample(n=sample_size, random_state=42)

## Reset the index of the shuffled sample

flights_sample.reset_index(drop=True, inplace=True)

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.